###################################################################       
#Script Name    :                                                                                              
#Description    :                                                                                 
#Args           :                                                                                           
#Author         : Nikhil Rao in R, converted to Python by Nor Raymond                                              
#Email          : nraymond@appen.com                                          
###################################################################

In [29]:
import os
import glob 
import pandas as pd
import numpy as np
import yaml
from IPython.core.display import display, HTML

In [35]:
# Function to load yaml configuration file
def load_config(config_name):
    with open(os.path.join(config_path, config_name), 'r') as file:
        config = yaml.safe_load(file)

    return config

config_path = "conf/base"

try:
    
    # load yaml catalog configuration file
    config = load_config("catalog.yml")

    os.chdir(config["project_path"])
    root_path = os.getcwd()
    
except:
    
    os.chdir('..')
    # load yaml catalog configuration file
    config = load_config("catalog.yml")

    os.chdir(config["project_path"])
    root_path = os.getcwd()

In [36]:
# import data_cleaning module
import src.data.data_cleaning as data_cleaning

### Functions to initialize data ingestion

In [37]:
def group_files_by_language(data_path, files, file_initials):
    
    file_groups = {}  
    for x in files:  
        key = x.split('_')[0] #x[:16] # The key is the first 16 characters of the file name
        group = file_groups.get(key,[])
        group.append(x)  
        file_groups[key] = group
                
    return file_groups

def create_file_exists_df(files, file_initials):
    
    checker = []
    file_exists = []
    for fname in files:
        for key in file_initials:
            if key in fname:
                file_exists.append((key, fname))

    file_exists = pd.DataFrame(file_exists, columns =['Keyword', 'Filename'])
    
    return file_exists

def data_ingestion_initialize(root_path, run_value, run_value_2):
    
    # Function to load yaml configuration file
    def load_config(config_name):
        with open(os.path.join(config_path, config_name), 'r') as file:
            config = yaml.safe_load(file)

        return config

    # load yaml catalog configuration file
    config = load_config("catalog.yml")
    
    # define reference file paths
    ref_path = os.path.join(root_path, config["data_path"]["ref"])
    ref_filepath = os.path.join(ref_path, config["filenames"]["rc_col_ref"])
    ref_data = pd.read_excel(io = ref_filepath, sheet_name="threshold_raters", header=None)
    
    if len(ref_data) != 0:
        ref_data_cols = ref_data[0].tolist()
    else:
        ref_data_cols = []

    print("Initialize data ingestion and file checking...\n")
    
    if run_value == 'Deployment':
        
        # define data input paths
        data_path = os.path.join(root_path, config["data_path"]["output"], 'Deployment')
        survey_path = ''
        
    else:
        
        # define data input paths
        data_path = os.path.join(root_path, config["data_path"]["output"], run_value, run_value_2)
        survey_path = os.path.join(root_path, config["data_path"]["survey"])
        post_survey_path = os.path.join(root_path, config["data_path"]["post_survey"])
        
    # get the list of files in raw folder
    files = os.listdir(data_path)
    files = [f for f in files if f[-5:] == '.xlsx']
    
    file_initials = ['RC', 'Vocab_1', 'Vocab_2']

    languages = []
    for file in files:
        for file_initial in file_initials:        
            lang = file.split('_' + file_initial)[0]
        if not lang.endswith((".xlsx")):
            languages.append(lang)
    
    languages = pd.DataFrame(languages, columns = ['Language'])
    
    file_groups = group_files_by_language(data_path, files, file_initials)
    
    file_exists = create_file_exists_df(files, file_initials)
    
    return data_path, files, languages, file_groups, file_exists, ref_data_cols, survey_path, post_survey_path
       

### Functions for data processing - DEPLOY

In [38]:
file_initials = ['RC', 'Vocab_1', 'Vocab_2']

def obtain_file_summary_df(file_initials, file_exists, data_path):
    
    df_summary = []
    for k in file_initials:
        selected_files = file_exists[file_exists['Keyword'] == k] 
        selected_filenames = selected_files['Filename'].tolist()

        df = pd.DataFrame()
        for f in selected_filenames:
            data = pd.read_excel(os.path.join(data_path, f), 'Summary')
            df = df.append(data)

        df_summary.append(df)    
        
    return df_summary

def obtain_file_data_df(file_initials, file_exists, data_path):
    
    df_data = []
    for k in file_initials:
        selected_files = file_exists[file_exists['Keyword'] == k] 
        selected_filenames = selected_files['Filename'].tolist()

        df = pd.DataFrame()
        for f in selected_filenames:
            data = pd.read_excel(os.path.join(data_path, f), 'Data')
            df = df.append(data)

        df_data.append(df)  
        
    return df_data

def obtain_distinct_raters(df_summary, ref_data_cols):

    r1 = df_summary[0] # Joined data for Summary sheet from RC 
    r2 = df_summary[1] # Joined data for Summary page from Vocab_1 
    r3 = df_summary[2] # Joined data for Summary page from Vocab_2 
             
    raters = pd.concat([r1,r2,r3], ignore_index=True)
    raters = raters[['_worker_id', 'Grouping', 'Market', 'Language']]
    raters = raters.drop_duplicates()
    
    if len(ref_data_cols) != 0:
        
        threshold_raters = ref_data_cols
        raters = raters[raters['_worker_id'].isin(threshold_raters)]
    
    # obtain languages from r1 and create a dataframe
    languages = r1.Language.unique().tolist()
    languages = pd.DataFrame(languages, columns = ['Language'])
    
    return raters, r1, r2, r3, languages

def merge_raters_to_df_data(df_data, raters):

    rc = df_data[0] # Joined data for Data sheet from RC 
    v1 = df_data[1] # Joined data for Data page from Vocab_1 
    v2 = df_data[2] # Joined data for Data page from Vocab_2 
    
    # Merge raters to v1, v2, and rc
    rc = pd.merge(rc, raters,  how='left', on=['_worker_id', 'Language'])
    v1 = pd.merge(v1, raters,  how='left', on=['_worker_id', 'Language'])
    v2 = pd.merge(v2, raters,  how='left', on=['_worker_id', 'Language'])
    
    # Convert _created_at and _started_at to date-time
    rc[['_created_at','_started_at']] = rc[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    v1[['_created_at','_started_at']] = v1[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    v2[['_created_at','_started_at']] = v2[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')

    return rc, v1, v2


### Functions for data processing - PILOT

In [39]:
def pilot_selection(root_path, config):

##  read pilot variation data from reference_check.xlsx file
    ref_path = os.path.join(root_path, config["data_path"]["ref"])
    ref_filepath = os.path.join(ref_path, config["filenames"]["rc_col_ref"])
    pilot_variation = pd.read_excel(io = ref_filepath, sheet_name="pilot_variation")
    
    print("\n")
    print(pilot_variation)
    
    while True:
        try:
            
            pilot_index = int(input("\nPlease select the number of the pilot variation: "))
            if pilot_index < min(pilot_variation.index) or pilot_index > max(pilot_variation.index):
                print(f"\nYou must enter numbers between {min(pilot_variation.index)} - {max(pilot_variation.index)}... Please try again")
                continue
            elif pilot_index == "":
                print("\nYou must enter any numbers")
                continue
            else:
                print(f"\nYou have selected {pilot_index} for '{pilot_variation.iloc[pilot_index, 1]}'\n")
                pilot_selected = pilot_variation.iloc[pilot_index, 0]
                pilot_var_selected = pilot_variation.iloc[pilot_index, 1]
                break

        except ValueError:
            print(f"\nYou must enter numerical values only... Please try again")
            continue
        else:
            break
    
    return pilot_variation, pilot_selected, pilot_var_selected

In [40]:
def survey_selection(root_path, config):

    survey_path = os.path.join(root_path, config["data_path"]["survey"])
       
    # get the list of files in raw folder
    files = os.listdir(survey_path)
    files = [f for f in files if f[-5:] == '.xlsx']
    
    survey_files = pd.DataFrame(files, columns = ['Survey Filename'])
    
    print(survey_files)
    
    while True:
        try:
            
            survey_index = int(input("\nPlease select the number of the survey filename for your pilot run: "))
            if survey_index < min(survey_files.index) or survey_index > max(survey_files.index):
                print(f"\nYou must enter numbers between {min(survey_files.index)} - {max(survey_files.index)}... Please try again")
                continue
            elif survey_index == "":
                print("\nYou must enter any numbers")
                continue
            else:
                print(f"\nYou have selected {survey_index} for '{survey_files.iloc[survey_index, 0]}'\n")
                survey_selected = survey_files.iloc[survey_index, 0]
                break

        except ValueError:
            print(f"\nYou must enter numerical values only... Please try again")
            continue
        else:
            break
    
    return survey_selected, survey_files

#### Functions for common pilot operations

In [41]:
def obtain_survey_fluency(survey_data):
    
    fluency = []
    for opt in survey_data['31_language_1']:
        if opt == 'over_15_years' :
            fluency.append('Fluent')
        elif opt == '1015_years' :
            fluency.append('Fluent')
        elif opt == '510_years' :
            fluency.append('Intermediate')
        elif opt == '03_years' :
            fluency.append('Not Fluent')   
        else:
            fluency.append('') 
            
    return fluency

def obtain_survey_data(survey_path, survey_selected):
         
    survey_data = pd.read_excel(os.path.join(survey_path, survey_selected), 'Sheet1')
    try:
        survey_data = survey_data.drop('Unnamed: 42', axis = 1)
        survey_data[['_created_at','_started_at']] = survey_data[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
        survey_data = survey_data.rename(columns = {"_created_at" : "survey_created_at", "_started_at" : "survey_started_at"})
        survey_data = survey_data[['_worker_id', '31_language_1', 'survey_created_at', 'survey_started_at']]
        survey_data['Fluency'] = obtain_survey_fluency(survey_data)
    except:
        survey_data = survey_data
        survey_data[['_created_at','_started_at']] = survey_data[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
        survey_data = survey_data.rename(columns = {"_created_at" : "survey_created_at", "_started_at" : "survey_started_at"})
        survey_data = survey_data[['_worker_id', '31_language_1', 'survey_created_at', 'survey_started_at']]
        survey_data['Fluency'] = obtain_survey_fluency(survey_data)
            
    return survey_data

def merge_to_survey_data(df_data, raters, survey_data):
    
    rc = df_data[0] # Joined data for Data sheet from RC 
    v1 = df_data[1] # Joined data for Data page from Vocab_1 
    v2 = df_data[2] # Joined data for Data page from Vocab_2 
    
    # Merge raters data to v1, v2, and rc
    rc = pd.merge(rc, raters,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v1 = pd.merge(v1, raters,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v2 = pd.merge(v2, raters,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    
    # Merge raters data to v1, v2, and rc
    rc = pd.merge(rc, survey_data,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v1 = pd.merge(v1, survey_data,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v2 = pd.merge(v2, survey_data,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    
    # Drop duplicate cols
    rc = rc.drop(['Language_y', 'Market'], axis = 1)
    v1 = v1.drop(['Language_y', 'Market'], axis = 1)
    v2 = v2.drop(['Language_y', 'Market'], axis = 1)
    
    rc = rc.rename(columns = {"Language_x":"Language"})
    v1 = v1.rename(columns = {"Language_x":"Language"})
    v2 = v2.rename(columns = {"Language_x":"Language"})
    
    # Convert _created_at and _started_at to date-time
    rc[['_created_at','_started_at']] = rc[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    v1[['_created_at','_started_at']] = v1[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    v2[['_created_at','_started_at']] = v2[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    
    return rc, v1, v2


#### Functions for varied pilot operations

In [62]:
## Create and merge to post survey data
# Note: functions relevant to Pilot 1A-1B
def obtain_post_survey_data(post_survey_path, pilot_selected, pilot_var_selected):

    '''
    Valid for Pilot 1A-1B
    '''
        
    post_survey_path = os.path.join(root_path, post_survey_path)
    post_survey_data = pd.read_excel(os.path.join(post_survey_path, pilot_selected, pilot_var_selected, 'Post Assessment Survey.xlsx'), 'Sheet1')
    
    postsurvey = post_survey_data[['_worker_id', 'question_1', 'question_1_1', 'question_2', 'question_2_1', 'question_3', 'question_3_1',
                                   'question_4', 'question_4_1', 'question_5', 'question_5_1', 'question_6', 'question_6_1']]
    
    postsurvey.columns = ["_worker_id",
                          "Did you feel any questions or content was inappropriate or offensive?",
                          "If you answered yes, please identify which question? Please explain",
                          "Did you notice any issues with the language? i.e., grammar, spelling, archaic terminology?",
                          "If so, what were the issues?",
                          "Did you feel you had enough time to complete the assessment in one sitting?",
                          "If so, did you feel pressured for time? Please share your thoughts and experience.",
                          "Did you feel there was a good range of difficulty in the questions?",
                          "Please explain your thoughts? Any details you can offer are greatly appreciated.",
                          "Did you use Google search / Google Translate / Word Reference / Linguee / any other sources to help you complete the test?",
                          "If so, please list what you used, and explain why you chose to use them?",
                          "If you did not finish the full assessment, we would love to know why. Did you find it too time consuming? Did you get interrupted?",
                          "If so, any information you can provide is helpful."]
    
    return postsurvey

def obtain_tenure_grouping(dfs, pilot_var_selected):
    
    '''
    Valid for Pilot 1A-1B if Grouping == GT -> Google Translate Control Group
    Valid for Pilot 1C - Grouping == GT -> GT
    '''
       
    tenure = []
    for opt in dfs['Grouping']:
        if opt == 'Pilot 1A' :
            tenure.append('90+ days')
        elif opt == 'Pilot 1B' :
            tenure.append('1 - 90 days')
        elif opt == 'GT' :
            if pilot_var_selected == 'Pilot 1A-1B':
                tenure.append('Google Translate Control Group')
            elif pilot_var_selected == 'Pilot 1C':
                tenure.append('GT')
        else:
            tenure.append('') 
            
    return tenure

def merge_to_post_survey_data(rc, v1, v2, postsurvey):
    
    '''
    Valid for Pilot 1A-1B
    '''
    
    # Merge postsurvey data to v1, v2, and rc
    rc = pd.merge(rc, postsurvey,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v1 = pd.merge(v1, postsurvey,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v2 = pd.merge(v2, postsurvey,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    
    rc, v1, v2 = fluency_categorization_group_3(rc, v1, v2)
    
        
    rc['Tenure'] = obtain_tenure_grouping(rc, pilot_var_selected)
    v1['Tenure'] = obtain_tenure_grouping(v1, pilot_var_selected)    
    v2['Tenure'] = obtain_tenure_grouping(v2, pilot_var_selected)    
    
    return rc, v1, v2


# Fluency categorization functions
# Note: missing group 3A-A , 2A-A has to be manual

def fluency_categorization_group_1(rc, v1, v2):
    
    '''
    Valid for 1C , 1D , 2A
    '''
    
    rc['Fluency'] = np.where(rc['Grouping'] == 'GT', 'GT', rc['Fluency'])
    v1['Fluency'] = np.where(v1['Grouping'] == 'GT', 'GT', v1['Fluency'])
    v2['Fluency'] = np.where(v2['Grouping'] == 'GT', 'GT', v2['Fluency'])
    
    rc = rc.dropna(subset=['Fluency'])
    v1 = v1.dropna(subset=['Fluency'])
    v2 = v2.dropna(subset=['Fluency'])
    
    return rc, v1, v2

def fluency_categorization_group_2(rc, v1, v2):
    
    '''
    Valid for 1E , 2D , 2B-A , 3A
    '''
    
    rc['Fluency'] = np.where(rc['Grouping'] == 'GT', 'GT', rc['Fluency'])
    v1['Fluency'] = np.where(v1['Grouping'] == 'GT', 'GT', v1['Fluency'])
    v2['Fluency'] = np.where(v2['Grouping'] == 'GT', 'GT', v2['Fluency'])
    
    rc['Fluency'] = np.where(rc['Fluency'].isna(), 'Fluent', rc['Fluency'])
    v1['Fluency'] = np.where(v1['Fluency'].isna(), 'Fluent', v1['Fluency'])
    v2['Fluency'] = np.where(v2['Fluency'].isna(), 'Fluent', v2['Fluency'])
    
    return rc, v1, v2

def fluency_categorization_group_3(rc, v1, v2):
    
    '''
    Valid for 1A-1B
    '''
    
    rc['Fluency'] = np.where(rc['31_language_1'].isna(), 'Not Categorized', rc['Fluency'])
    v1['Fluency'] = np.where(v1['31_language_1'].isna(), 'Not Categorized', v1['Fluency'])
    v2['Fluency'] = np.where(v2['31_language_1'].isna(), 'Not Categorized', v2['Fluency'])
    
    rc['Fluency'] = np.where(rc['Grouping'] == 'GT', 'GT', rc['Fluency'])
    v1['Fluency'] = np.where(v1['Grouping'] == 'GT', 'GT', v1['Fluency'])
    v2['Fluency'] = np.where(v2['Grouping'] == 'GT', 'GT', v2['Fluency'])
    
    return rc, v1, v2





In [76]:
def main():

    file_initials = ['RC', 'Vocab_1', 'Vocab_2']
    
    language, market, run_value, run_value_2  = data_cleaning.main()
    
    data_path, files, languages, file_groups, file_exists, ref_data_cols, survey_path, post_survey_path = data_ingestion_initialize(root_path, run_value, run_value_2)
    
    df_summary = obtain_file_summary_df(file_initials, file_exists, data_path)
    df_data = obtain_file_data_df(file_initials, file_exists, data_path)
    raters, r1, r2, r3, languages =  obtain_distinct_raters(df_summary, ref_data_cols)
        
    if run_value == 'Deployment':
        
        rc, v1, v2 = merge_raters_to_df_data(df_data, raters)
        
    else: 
        
        pilot_variation, pilot_selected, pilot_var_selected = pilot_selection(root_path, config)
        survey_selected, survey_files = survey_selection(root_path, config)
        survey_data = obtain_survey_data(survey_path, survey_selected)
        rc, v1, v2 = merge_to_survey_data(df_data, raters, survey_data)
        
        if pilot_var_selected == 'Pilot 1A-1B':
            
            postsurvey = obtain_post_survey_data(post_survey_path, pilot_selected, pilot_var_selected)
            rc, v1, v2 = merge_to_post_survey_data(rc, v1, v2, postsurvey)
            
        elif pilot_var_selected == 'Pilot 1C':
            
            rc, v1, v2 = fluency_categorization_group_1(rc, v1, v2)
            rc['Tenure'] = obtain_tenure_grouping(rc, pilot_var_selected)
            v1['Tenure'] = obtain_tenure_grouping(v1, pilot_var_selected)    
            v2['Tenure'] = obtain_tenure_grouping(v2, pilot_var_selected) 
            
        elif pilot_var_selected == 'Pilot 1D' or pilot_var_selected == 'Pilot 2A':
            
            rc, v1, v2 = fluency_categorization_group_1(rc, v1, v2)    
            
        elif pilot_var_selected == 'Pilot 1E':
            
            rc, v1, v2 = fluency_categorization_group_2(rc, v1, v2)   
 
    return rc, v1, v2, survey_data, r1, r2, r3

#    return raters, r1, r2, r3, languages, rc, v1, v2, run_value, run_value_2, survey_selected, survey_files  

if __name__ == "__main__":

#     raters, r1, r2, r3, languages, rc, v1, v2, run_value, run_value_2, survey_selected, survey_files  = main()
#     print(languages)

    rc, v1, v2, survey_data, r1, r2, r3 = main()
    
    print('\nAutomated data processing completed.')

Initialize data ingestion and file checking...

PASS: All files exists!



Please input the type of run e.g. Deployment, Pilot 1, Pilot 2, Pilot 3 .... etc.:  Pilot 2



Run type: Pilot 2



Please input the pilot subfolder name e.g. Pilot 1A, Pilot 2C, Pilot 3A-B .... etc.:  Pilot 2A



Pilot subfolder: Pilot 2A


...


Starting automated data cleaning....

Dataframe created from RC file
Language and Market columns and values inserted to 'Summary' sheet
Language column and values inserted to 'Data' sheet
Missing columns inserted into 'Data' sheet.

Preview cleaned datasets:



df_summary_cleaned


             Language Market  _worker_id  Score  Percentage Grouping
0  Chinese-Simplified  ZH-ZH    45699754     24    1.000000  Pilot 2
1  Chinese-Simplified  ZH-ZH    45701425     21    0.875000  Pilot 2
2  Chinese-Simplified  ZH-ZH    45746251     24    1.000000  Pilot 2
3  Chinese-Simplified  ZH-ZH    45797032     23    0.958333  Pilot 2
4  Chinese-Simplified  ZH-ZH    46047420     18    0.750000       GT


df_data_cleaned


             Language           _id question_no_1 question_no_2 question_no_3  \
0  Chinese-Simplified  5.765947e+09             b             a             a   
1  Chinese-Simplified  5.765995e+09             b             a             a   
2  Chinese-Simplified  5.766195e+09    


Please select the number of the pilot variation:  4



You have selected 4 for 'Pilot 2A'

               Survey Filename
0  Survey Pilot 1A and 1B.xlsx
1          Survey Pilot 2.xlsx
2    Survey Pilot 2 and 3.xlsx



Please select the number of the survey filename for your pilot run:  1



You have selected 1 for 'Survey Pilot 2.xlsx'


Automated data processing completed.


In [79]:
v2

,Language,_id,rater_answer,a_domain,a_register,b_domain,b_register,difficulty,familiarity,question_,...,_ip,Answer,Alternate Answer,Score,Answers,Grouping,31_language_1,survey_created_at,survey_started_at,Fluency
0,Chinese-Simplified,5765924548,a_is_more_specific_than_b,aviation,slang/informal,aviation,formal,hard,unfamiliar,1,...,45.135.186.84,a_is_more_specific_than_b,0,1,a_is_more_specific_than_b;0,Pilot 2,over_15_years,2020-10-07 01:00:39,2020-10-07 00:59:07,Fluent
1,Chinese-Simplified,5765972394,a_is_more_specific_than_b,aviation,slang/informal,aviation,formal,hard,unfamiliar,1,...,103.136.251.48,a_is_more_specific_than_b,0,1,a_is_more_specific_than_b;0,Pilot 2,over_15_years,2020-10-07 11:54:18,2020-10-07 11:49:30,Fluent
2,Chinese-Simplified,5766132493,a_is_more_specific_than_b,aviation,slang/informal,aviation,formal,hard,unfamiliar,1,...,114.240.58.121,a_is_more_specific_than_b,0,1,a_is_more_specific_than_b;0,Pilot 2,over_15_years,2020-10-07 08:06:48,2020-10-07 08:03:12,Fluent
3,Chinese-Simplified,5766307171,a_is_more_specific_than_b,aviation,slang/informal,aviation,formal,hard,unfamiliar,1,...,210.3.145.194,a_is_more_specific_than_b,0,1,a_is_more_specific_than_b;0,Pilot 2,over_15_years,2020-10-07 09:00:43,2020-10-07 08:56:00,Fluent
4,Chinese-Simplified,5766689403,a_is_more_specific_than_b,aviation,slang/informal,aviation,formal,hard,unfamiliar,1,...,91.220.202.202,a_is_more_specific_than_b,0,1,a_is_more_specific_than_b;0,Pilot 2,over_15_years,2020-10-07 01:10:43,2020-10-07 01:06:33,Fluent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,Chinese-Simplified,5766777381,b_is_more_specific_than_a,music,neutral,music,neutral,medium,familiar,80,...,104.163.191.52,b_is_more_specific_than_a,0,1,b_is_more_specific_than_a;0,Pilot 2,over_15_years,2020-10-07 15:57:37,2020-10-07 15:54:09,Fluent
876,Chinese-Simplified,5767073413,b_is_more_specific_than_a,music,neutral,music,neutral,medium,familiar,80,...,38.94.109.21,b_is_more_specific_than_a,0,1,b_is_more_specific_than_a;0,Pilot 2,over_15_years,2020-10-06 21:49:40,2020-10-06 21:48:15,Fluent
877,Chinese-Simplified,5769869916,b_is_more_specific_than_a,music,neutral,music,neutral,medium,familiar,80,...,49.144.1.167,b_is_more_specific_than_a,0,1,b_is_more_specific_than_a;0,GT,NaN,NaT,NaT,GT
878,Chinese-Simplified,5769891239,b_is_more_specific_than_a,music,neutral,music,neutral,medium,familiar,80,...,99.150.146.137,b_is_more_specific_than_a,0,1,b_is_more_specific_than_a;0,GT,NaN,NaT,NaT,GT


In [ ]:
##  create manual pilot variation data    
#     data = {'Pilot': ['Pilot 1', 'Pilot 1', 'Pilot 1', 'Pilot 1', 'Pilot 2', 'Pilot 2','Pilot 2', 'Pilot 2', 'Pilot 3', 'Pilot 3'],
#             'Variation': ['Pilot 1A-1B', 'Pilot 1C', 'Pilot 1D', 'Pilot 1E', 'Pilot 2A', 'Pilot 2A-A', 'Pilot 2B-A', 'Pilot 2D', 'Pilot 3A', 'Pilot 3A-A']}
    
#     pilot_variation = pd.DataFrame.from_records(data)